In [4]:
import dash_bootstrap_components as dbc
from dash import Input, Output, dcc, html
import dash
import warnings 
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd
from shapely import wkb, speedups
import geopandas as gpd
speedups.disable()
import plotly.express as px
import plotly.graph_objects as go
import pymssql
from joblib import load

database = 'Pushing-P-DB'
table2 = 'dbo.NJ_Census_Tracts'
table3 = 'dbo.NJ_County_Info'
table = 'dbo.Live_Feed'
user = 'pushing_p'
password  = 't3stP@ssword'
server = 'gen10-data-fundamentals-22-02-sql-server.database.windows.net'

col_dict = {
'NUI, Under 6 years': '% Uninsured, Under 6 years',
'NUI, 6-18 years': '% Uninsured, 6-18 years',
'NUI, 19 to 25 years': '% Uninsured, 19 to 25 years',
'NUI, 26 to 34 years': '% Uninsured, 26 to 34 years',
'NUI, 35 to 44 years': '% Uninsured, 35 to 44 years',
'NUI, 45 to 54 years': '% Uninsured, 45 to 54 years',
'NUI, 55 to 64 years': '% Uninsured, 55 to 64 years',
'NUI, 65 years and older': '% Uninsured, 65 years and older',
'NUI, Men': '% Uninsured, Men',
'NUI, Women': '% Uninsured, Women',
'NUI, White': '% Uninsured, White',
'NUI, African American': '% Uninsured, African American',
'NUI, American Indian & Alaska Natives': '% Uninsured, American Indian & Alaska Natives',
'NUI, Asians': '% Uninsured, Asians',
'NUI, Native Hawaiians & Pacific Islanders': '% Uninsured, Native Hawaiians & Pacific Islanders',
'NUI, Other Races': '% Uninsured, Other Races'}

percent_cols = list(col_dict.values())
numeric_cols = list(col_dict.keys())


def Get_Data(table):
    try:
        conn = pymssql.connect(server,user,password,database)

        # Query select all rows from SQL tables to insert into their respective DataFrames
        query = f'SELECT * FROM {table}'

        df = pd.read_sql(query, conn)

        def convert_to_polygon(hex):
            return wkb.loads(hex, hex=True)

        df['geometry'] = df['geometry'].apply(convert_to_polygon)
        
        df.dropna(inplace=True)

        df[['City Population','Uninsured Population']] = df[['City Population','Uninsured Population']].astype(int)
        
        df = pd.DataFrame(df)
        df['% Uninsured'] = (df['Uninsured Population']/df['City Population']) * 100
        df['% Uninsured'] = df['% Uninsured'].round(decimals = 2)
        df['Insured Population'] = df['City Population'] - df['Uninsured Population']
        df['Insured Population'] = df['Insured Population'].round(decimals = 2)

        df[numeric_cols] = df[numeric_cols].astype(float)

        df[percent_cols] = df[numeric_cols].div(df['Uninsured Population'], axis=0)

        df[percent_cols] = df[percent_cols].apply(lambda x: x * 100).round(decimals = 2)

        # converting to geopandas dataframe
        df = gpd.GeoDataFrame(df, geometry=df['geometry'], crs = 'epsg:4269')

        return df

    except Exception as e:
        raise e


df = Get_Data(table)   # Live_Feed table
tracts = Get_Data(table2)  # Census Tract Data
counties = Get_Data(table3)  # County Data
results = load('areal interpolation joblib.model')
results[percent_cols] = results[percent_cols].astype(float).round(decimals = 2)
results['% Uninsured'] = results['% Uninsured'].astype(float).round(decimals = 2)

def Make_Plots(df):
    
    county_name = df.County[0]

    # Here we set the colors I will be using for the visualizations
    colors = {
        'background': '#111111',
        'text': '#7FDBFF'
    }

    
# Figure 1. BAR % UNINSURED VS. AGE
    df1= df[['State','County','City','NUI, Under 6 years', 'NUI, 6-18 years', 'NUI, 19 to 25 years',
    'NUI, 26 to 34 years', 'NUI, 35 to 44 years', 'NUI, 45 to 54 years',
    'NUI, 55 to 64 years', 'NUI, 65 years and older','% Uninsured']].copy()

    df1 = df1 [['NUI, Under 6 years', 'NUI, 6-18 years', 'NUI, 19 to 25 years',
    'NUI, 26 to 34 years', 'NUI, 35 to 44 years', 'NUI, 45 to 54 years',
    'NUI, 55 to 64 years', 'NUI, 65 years and older']].sum()

    df1 = pd.DataFrame(df1)

    df1 = df1.reset_index()

    df1.rename(columns = {'index':'Age', 0:'Number Of Uninsured'}, inplace = True)

    State_Uninsured = df['Uninsured Population'].sum()

    df1['Uninsured Rate By Age'] = df1['Number Of Uninsured']/State_Uninsured * 100

    df1['Uninsured Rate By Age'] = df1['Uninsured Rate By Age'].round(decimals = 2)


    fig_1 = px.bar(df1, x='Age', y='Uninsured Rate By Age',color_discrete_sequence=["green"], title= f'% Uninsured By Age For {county_name}', height = 500, width = 700)
    fig_1.update_layout(xaxis={'categoryorder':'total descending'})


    fig_1.update_layout(
        title=dict(x=0.5), #set title in the center
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        width = 600
    )
   
    # Figure 2. BAR % UNINSURED VS. RACE
   
    df2= df[['NUI, White', 'NUI, African American','NUI, American Indian & Alaska Natives',
    'NUI, Asians','NUI, Native Hawaiians & Pacific Islanders', 'NUI, Other Races','% Uninsured']].copy()

    df2.rename(columns = {'NUI, American Indian & Alaska Natives':'NUI, American I. & Alaska N.', 'NUI, Native Hawaiians & Pacific Islanders':'NUI, Native H. & Pacific I.'}, inplace = True)

    df2 = df2[['NUI, White', 'NUI, African American','NUI, American I. & Alaska N.',
    'NUI, Asians','NUI, Native H. & Pacific I.', 'NUI, Other Races']].sum()

    df2 = pd.DataFrame(df2)

    df2 = df2.reset_index()

    df2.rename(columns = {'index':'Race', 0:'Number Of Uninsured'}, inplace = True)

    State_Uninsured = df['Uninsured Population'].sum()

    df2['Uninsured Rate By Race'] = df2['Number Of Uninsured']/State_Uninsured * 100

    df2['Uninsured Rate By Race'] = df2['Uninsured Rate By Race'].round(decimals = 2)


    fig_2 = px.bar(df2, x='Race', y='Uninsured Rate By Race',color_discrete_sequence=["blue"], title= f'% Uninsured By Race For {county_name}', height = 500, width = 700)
    fig_2.update_layout(xaxis={'categoryorder':'total descending'})

    # this is where I center the title for the visualization
    fig_2.update_layout(
        title=dict(x=0.45), #set title in the center
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        width = 600,
        title_font=dict(size= 15)
 
    )


# Figure 3. TABLE TOP 10 UNINSURED CITIES VISUAL PLACE
    df3= df[['City','Uninsured Population','% Uninsured']].copy()

    df3 = df3.sort_values(by='% Uninsured', ascending = False)

    df3 = df3.head(10)

    fig_3 = go.Figure(data=[go.Table(
        header=dict(values=df3.columns,
                line_color='darkslategray',
                font_color='#7FDBFF',
                fill_color='Black',
                align='left'),
        cells=dict(values=[df3.City, df3['Uninsured Population'], df3['% Uninsured']],
                line_color='darkslategray',
                fill_color='White',
                font_color='black',
                align='left'))
    ])

    fig_3.update_layout(
        title = f'Top 10 Uninsured Cities by Rate in {county_name}',
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        
        title_font_color = '#7FDBFF'
    )

    # fig_3.update_layout(title_font_color='7FDBFF')

    fig_3.update_layout(
        title=dict(x=0.5)
    )

# Figure 4. DONUTS VISUAL ONE PLACE
    df4 = df.copy()

    df4 = df4[['Insured Population', 'Uninsured Population']].sum()

    df4 = pd.DataFrame(df4)

    df4 = df4.reset_index()

    df4.rename(columns = {'index':'Variables', 0:'Sum'}, inplace = True)


    colors = ['lime','orange']
    fig_4 = go.Figure(data = go.Pie(values = df4.Sum, 
                            labels = df4.Variables, hole = 0.6,
                            marker_colors = colors,
                    ))
    fig_4.update_traces(
                    title_font=dict(size=30,family='Verdana', 
                                    color='white'),
                                    hoverinfo='label+percent',
                                    textinfo='percent', 
                                    textfont_size=20,
                    )

    fig_4.update_layout(legend=dict(y=1.1, x = 0.8), title = f'Insured and Uninsured Rates in {county_name}',
    paper_bgcolor = '#111111', font = {'color': "#7FDBFF", 'family': "Arial"})
    fig_4.update_layout(title=dict(x=0.5), title_font=dict(size= 20)) #set title in the center)


    return  fig_1, fig_2, fig_3, fig_4


fig_1, fig_2, fig_3, fig_4 = Make_Plots(df)


# Make Maps on Second Page

# Default variable is % Uninsured
def Make_Plots_Maps(counties, cities, tracts, variable = '% Uninsured'):

    county_name = cities.at[0, 'County']
    counties = counties.loc[counties['County'] == county_name]
    counties = counties.set_index('County')

    px.set_mapbox_access_token('pk.eyJ1IjoiYWhhZGg3NjIiLCJhIjoiY2wzaTBqbnQ2MGU2cjNqbzZpNXFiZHk1eSJ9.UTxbELXv9gk6QiowY1VgqA')
    
    # Get Centerpoint of County
    lon = float(counties.centroid.x.values[0])
    lat = float(counties.centroid.y.values[0])

    cities = cities.set_index('City')

    # Predicted Census Tracts Map (Machine Learning Model)
    model = results.loc[results['County'] == county_name]
    model = model.set_index('Census Tract')

    # Figure 5. MAP OF PREDICTED CENSUS TRACT FROM ML MODEL
    fig_5 = px.choropleth_mapbox(model, geojson=model.__geo_interface__, locations=model.index, color = variable,
                            mapbox_style="outdoors", color_continuous_scale='Viridis_r',
                            zoom=7.8, center = {"lat": lat , "lon": lon},
                            )

    fig_5.update_layout(
        margin=dict(l=35, r=35, t=35, b=35),
        title = f'{county_name} Census Tracts (Predicted)',  title_x = 0.50,
        height = 600,
        paper_bgcolor="#111111",
        font = {'color': "#7FDBFF", 'family': "Arial"})
    
    fig_5.layout.coloraxis.colorbar.title = ''

    fig_5['layout']['title']['font'] = dict(size=20)
    fig_5.update_geos(fitbounds="locations")

     # Predicted Census Tracts Map
    tracts = tracts.loc[tracts['County'] == county_name]
    tracts = tracts.set_index('Census Tract')

    # Figure 6. MAP OF ACTUAL CENSUS TRACT DATA
    fig_6 = px.choropleth_mapbox(tracts, geojson=tracts.__geo_interface__, locations=tracts.index, color = variable,
                            mapbox_style="outdoors", color_continuous_scale='Viridis_r',
                            zoom=7.8, center = {"lat": lat , "lon": lon},
                            )

    fig_6.update_layout(
        margin=dict(l=35, r=35, t=35, b=35),
        title = f'{county_name} Census Tracts (Actual)',  title_x = 0.50,
        height = 600,
        paper_bgcolor="#111111",
        font = {'color': "#7FDBFF", 'family': "Arial"})
    
    fig_6.layout.coloraxis.colorbar.title = ''


    fig_6['layout']['title']['font'] = dict(size=20)
    fig_6.update_geos(fitbounds="locations")


    return fig_5, fig_6


fig_5, fig_6 = Make_Plots_Maps(counties, df, tracts)


# CSS Styling

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#545853",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("", className="display-4", style = {'color':'#7FDBFF'}),
        html.Hr(),
        html.P(
            "Navigation Menu", className="lead", style = {'color':'#7FDBFF'}
        ),
        dbc.Nav(
            [
                dbc.NavLink("Demographics", href="/page-1", active=""),
                dbc.NavLink("Maps", href="/page-2", active="exact"),
                dbc.NavLink("About Us", href="/page-3", active="exact")

            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content], style={'backgroundColor':'#111111'})


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):

    # First Page
    if pathname == "/page-1" or pathname=="/":
        
        return[
               
        html.H1(children = 'Healthcare Demographic Data ', id = 'title', style={"text-align": "center", "color": "white"}),
               
        html.Div([

                    html.Div(
                    children = dcc.Graph(
                        id = '% UNINSURED VS AGE',
                        figure = fig_1,
                    ),
                    style={'width': '50%', 'display': 'inline-block'},    
                ),
                    html.Div(
                    children = dcc.Graph(
                        id = 'Race',
                        figure = fig_2,
                    ),
                    style={'width': '50%','display': 'inline-block'},
                ),
                     html.Div(
                    children = dcc.Graph(
                        id = 'Table',
                        figure = fig_3,
                    ),
                    style={'width': '50%', 'display': 'inline-block'},
                ),
                    html.Div(
                    children = dcc.Graph(
                        id = 'donut-1',
                        figure = fig_4,
                    ),
                    style={'width': '50%', 'display': 'inline-block'},
                ),

                    dcc.Interval(
                    id='interval-component',
                    interval= 2000, # in milliseconds (there will be an update once every 2 seconds)
                    n_intervals=0)],       
            className = 'double-graph'
        )
        ]        
             
    # Second Page
    elif pathname == "/page-2":

        return [
            
             html.H1(children = 'Machine Learning Maps' , id = 'title1', style={"text-align": "center", "color": "white"}),
            
            html.Div([

            # Dash Application Layout
            html.Div([
                dcc.Dropdown(['% Uninsured'] +percent_cols, '% Uninsured', id='dropdown',
                clearable = False,
                searchable = False),
                html.Div(id='dd-output-container')
            ]),
            
            html.Div( style={'padding': '20px'}),
           
            dcc.Graph(id = 'Predicted Census Tract Map',
            figure = fig_5, style={'width': '50%', 'display': 'inline-block'}),

            dcc.Graph(id = 'Actual Census Tract Map',
            figure = fig_6, style={'width': '50%', 'display': 'inline-block'}),

            dcc.Interval(
            id='interval-component2',
            interval= 2000, # in milliseconds (there will be an update once every minute)
            n_intervals=0)])]

        
    # About US Page 3
    elif pathname == "/page-3":

        return [html.Div([
            html.H1(
              children = "About Us", style={'fontSize': "30px",'textAlign': 'center','color':'#7FDBFF', 'backgroundColor':'#111111',}),

            html.P(['Our team here in Dev10 composed of Sharif, Ahad, Luis, Parth, and Marjea, have conducted extensive research in New Jersey dealing with health insurance and predicting the probability of an individual being insured depending on the target geography they find themselves in.', html.Br(),
            'We analyzed several demographics such as age and race to determine who a typical resident/individual would be in NJ. '
            'The machine learing model we used is called Spatial Area Interpolation. Spatial Area Interpolation is a technique for estimating values contained within target geographies with unknown values. Our target geographies were census geographical tract areas.', html.Br(),
            'Here in Dev10, we value the opportunity to make an impact in our world through technology to understand what is going on around us.', html.Br(),
            'To learn more, we invite you to take a look at the dashboard and reach out to us with any questions.'], className='my-class', id='my-p-element'),
            ], style={ 'backgroundColor':'#111111', 'color':'#7FDBFF', 'fontSize': 28})]


@app.callback(
Output("% UNINSURED VS AGE", "figure"),
Output("Race", "figure"),
Output("Table", "figure"),
Output("donut-1", "figure"),
Output('title', 'children'),
Input('interval-component', "n_intervals")
)


def update_charts(n):

    df = Get_Data(table)
    county_name = df.County[0]
    fig_1, fig_2, fig_3, fig_4 = Make_Plots(df)
    
    return fig_1, fig_2, fig_3, fig_4, ('Healthcare Demographic Data for ' + f'{county_name}')
   
    
@app.callback(
Output("Predicted Census Tract Map", "figure"),
Output("Actual Census Tract Map", "figure"),
Output('title1', 'children'),
Input('interval-component2', "n_intervals"),
Input("dropdown", "value")
)

def update_charts2(n, dropdown):
    
    df = Get_Data(table)
    county_name = df.County[0]
    fig_5, fig_6 = Make_Plots_Maps(counties, df, tracts, dropdown)

    return fig_5, fig_6, ('Machine Learning Maps for ' + f'{county_name}')


if __name__ == '__main__':
    app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2022 00:17:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2022 00:17:45] "GET /_dash-dependencies